In [1]:
import torch
print(torch.__version__)

1.7.1


In [2]:
from __future__ import print_function

import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision import models
import numpy
import dataset
import dataset_test
import random
from utils import *
from VGG_gru import FERANet
# from tensorboardX import SummaryWriter
import argparse
import subprocess
import math
import csv
import pandas as pd


## Loading Model

In [3]:
# model can be downloaded here: https://drive.google.com/drive/folders/1f17xgwvGaUpgXYBssocUNXDBgga-b3qp
preInitial = True
if preInitial == True:
	model = FERANet()
	model = Initial(model)
else:
	model = FERANet()

start loading vgg16 pre-fera2013 model...
finish loading vgg16 pre-fera2013 model!


In [4]:
processed_batches = 0
num_workers   = 0
torch.manual_seed(1)
torch.cuda.manual_seed(1)
random.seed(1)
torch.backends.cudnn.benchmark = True

kwargs = {'num_workers': num_workers, 'pin_memory': True}
model = model.cuda()


### MODEL PARAMETERS:

In [5]:
batch_size = 1
learning_rate = 0.0001
epochs = 20 
length = 32

use_Tensorboard = False

### TO CHANGE:

In [6]:
###### TO CHANGE #######
EXP_TYPE = 'Regression/'
# EXP_TYPE = 'Classification/'

optimizer = optim.SGD(model.parameters(), lr=learning_rate , momentum=0.9, weight_decay= 0.00005)

#### Change loss function in UTILS.py
# # loss_function = nn.CrossEntropyLoss()
# # loss_function = nn.L1Loss()
# loss_function = nn.MSELoss()
metric = RegressionMetric()

### TO KEEP:

In [7]:
def train(epoch,optimizer):
      
	train_loader = torch.utils.data.DataLoader(
		dataset.listDataset(trainlist,length = length,
					   shuffle=True,
					   train=True,
					   dataset = dataset),
		batch_size=batch_size, shuffle=False, **kwargs)


	for param_group in optimizer.param_groups:
		train_learning_rate = float(param_group['lr'])

	logging('epoch %d, processed %d samples, lr %f' % (epoch, epoch * len(train_loader.dataset), train_learning_rate))

	running_loss = 0.0

	model.train()

	for batch_idx, (data, label) in enumerate(train_loader):


		data = data.squeeze(0)
		data = Variable(data).cuda()

		label = Variable(label.float()).cuda()

		optimizer.zero_grad()

		output = model(data)

		loss = loss_function(output, label)

		running_loss += loss.data

		loss.backward()

		optimizer.step()

	if epoch %1 == 0:
		logging('Train Loss:{:.6f}'.format(running_loss))
        
	return running_loss        

In [8]:
def eval(epoch,metric):

	model.eval()
    
	test_loader = torch.utils.data.DataLoader(
		dataset.listDataset(vallist,length = length,       
					shuffle=False,
					train=False,
					dataset = dataset),
					batch_size=1, shuffle=False, **kwargs)

	for batch_idx, (data, label) in enumerate(test_loader):

		data = data.squeeze(0)
		Batch,T,C,H,W = data.size()

		data = Variable(data,volatile=True).cuda()

		label = Variable(label.float(),volatile=True).cuda()
		output = []
		for batch_index in range(Batch):
# 			print(batch_index)            
			output_feature = model(data[batch_index])
			output.append(output_feature)
		output = torch.mean(torch.cat(output), 0, keepdim=True) 
# 		print("Dua's output is: ", output)
		metric(output, label) 
		eval_loss = metric.value()
	if epoch %1 == 0:
		logging("Eval Loss: %f" % (eval_loss))
    
	return eval_loss

In [9]:
def test(epoch,metric):
    
	model.eval()
	result = []    
	test_loader = torch.utils.data.DataLoader(
		dataset_test.listDataset(testlist,length = length,
# 		dataset.listDataset(vallist,length = length,        
					shuffle=False,
					train=False,
					dataset = dataset),
					batch_size=1, shuffle=False, **kwargs)
    
	for batch_idx, (data, label, participant) in enumerate(test_loader):

		data = data.squeeze(0)

		Batch,T,C,H,W = data.size()
		data = Variable(data,volatile=True).cuda()

		label = Variable(label.float(),volatile=True).cuda()
# 		print('Participant name:',participant)
# 		print('Participant label:',int(label))
		output = []
		for batch_index in range(Batch):           
			output_feature = model(data[batch_index])
			output.append(output_feature)
		output = torch.mean(torch.cat(output), 0, keepdim=True)
# 		print("Participant output is: ", int(output))
		metric(output, label) 
		test_loss = metric.value()
# 		print("Participant loss is: ", test_loss)
		result.append([participant, int(label), int(output), test_loss])
	return result

In [10]:
COMB = ['BDI','AVEC','AVEC-BDI','AVEC-TEST','BDI-TEST']
NUM = ['1','2','3','4','5']
#Text file of image path for each of the above combinations
textMAIN = '/timo/datasets/Dua/GRU-test/Data-Combinations/'
textPATH = textMAIN+EXP_TYPE
backupdir     = '/timo/datasets/Dua/GRU-test/Saved-Models'
results_PATH = '/timo/datasets/Dua/GRU-test/Results/'
#Datasets Path (participants Path) AVEC + BDI
# path_d= '/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/dataset.py'
#Where to save trained models Path


In [11]:
# if os.path.isfile != True:
# 	subprocess.call(["python", path_d])

## Training+Evaluating 

In [12]:
for C in COMB:
    for N in NUM:
        train_val_loss = []
        trainlist=textPATH+C+'train'+N+'.txt'
        vallist=textPATH+C+'val'+N+'.txt'
        for epoch in range(1, epochs+1): 
            train_loss = train(epoch,optimizer)
            with torch.no_grad():
                eval_loss = eval(epoch,metric)
            train_val_loss.append([int(train_loss), int(eval_loss)]) 
        with open(results_PATH+C+'train_val'+N+'_loss.csv', 'w') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerows(train_val_loss)
        csvFile.close()
        torch.save(model.state_dict(),'%s/model_%s%s.pkl' % (backupdir,C,N))
        #add saving train +eval loss

2021-02-19 12:52:19 epoch 1, processed 36 samples, lr 0.000100
2021-02-19 12:52:31 Train Loss:4051.439453


/home/duaa/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/duaa/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


KeyboardInterrupt: 

## Testing Results

In [ ]:
for C in COMB:
    for N in NUM:
        testlist=textPATH+C+'test'+N+'.txt'
        model.load_state_dict(torch.load('%s/model_%s%s.pkl' % (backupdir,C,N)))
        with torch.no_grad():
            test_accuary = test(epoch,metric)
        with open(results_PATH+C+'test'+N+'Results.csv', 'w') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerows(test_accuary)
        csvFile.close()

In [ ]:
# data = pd.read_csv('/timo/datasets/Dua/GRU-test/Results/test.csv')
# data